# Tail Angle Data Parsing and Figure Creation

In [2]:
# base and metaprogramming imports
from __future__ import annotations

import datetime
import os
import shutil
import sys
import itertools as it
import pprint as pp
from typing import Optional
import warnings
import glob
import copy
import functools as ft

# analysis imports
import h5py
import numpy as np
import pandas as pd
import pandas
import yaml
from pandas.api.types import CategoricalDtype
import scipy
import scipy.ndimage
import skimage as ski
from tifffile.tifffile import imwrite

import ants
try:
    ants.from_numpy
except Exception:
    warnings.warn(f'`ants` module import failed.')
    ants = None
else:
    print('`ants` installed and imported successfully.')

# visualization and plotting imports
import napari
import napari.settings
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# caiman imports
import bokeh.plotting as bpl
import cv2
try:
    cv2.setNumThreads(0)
except Exception:
    pass
import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
bpl.output_notebook()

# ipynb-specific imports
try:
    import ipynbname
except Exception as e:
    warnings.warn(f'`ipynbname` module import failed with error: {e!r}')
    ipynbname = None
    
if ipynbname is not None:
    _file = ipynbname.path()
else:
    # manually set the full path to the current notebook below if `ipynbname` 
    # import fails
    raise RuntimeError('`ipynbname` import failed. Please set the `_file`'
                       'variable manually.')
    _file = ''
    
jobs_dir = os.path.split(_file)[0]
sources_dir = os.path.abspath(os.path.join(jobs_dir, '..'))
repo_dir = os.path.abspath(os.path.join(sources_dir, '..'))
config_dir = os.path.abspath(os.path.join(repo_dir, 'configs'))

# add sources directory to path
if sources_dir not in sys.path:
    sys.path.append(sources_dir)

# local imports
import c_swain_python_utils as csutils
from utilities import *
import imaging_dataset as imd
import prairie_view_imports as pvi

_basename = os.path.basename(_file)
_name = csutils.no_ext_basename(_file)

`ants` installed and imported successfully.


Loading BokehJS ...

In [3]:
load_path = (
    r'G:\c_swain\zf_correlative_microscopy'
    r'\analyzed_and_generated_data'
    r'\cs-ii-29_outputs'
    r'\bout_prop_df.csv')

bout_prop_df = csutils.load_from_disk(load_path)
bout_prop_df.head()

,experiment_id,run_id,fish_id,bout_index,bout_timestamp,bout_framestamp,bout_duration,bout_max_curvature,bout_max_angle_range,bout_duration_ms,unique_run_id,count_col,bout_time_bin
0,cs-ii-29,395,cs-ii-29B,0,23.637,5623,0.210,0.029499,104.740924,210.0,cs-ii-29_395,1,"(0, 60]"
1,cs-ii-29,395,cs-ii-29B,1,48.073,11463,0.250,0.097845,85.426512,250.0,cs-ii-29_395,1,"(0, 60]"
2,cs-ii-29,395,cs-ii-29B,2,48.502,11566,0.288,0.197285,139.522245,288.0,cs-ii-29_395,1,"(0, 60]"
3,cs-ii-29,395,cs-ii-29B,3,195.123,46548,0.198,0.035623,86.689587,198.0,cs-ii-29_395,1,"(180, 240]"
4,cs-ii-29,396,cs-ii-29B,0,48.291,12067,0.370,0.029650,70.336598,370.0,cs-ii-29_396,1,"(0, 60]"


In [5]:
pivot = (
    bout_prop_df.pivot_table(
        index='bout_time_bin', 
        columns=['fish_id', 'unique_run_id'], 
        values='count_col',
        aggfunc='count')
    .replace(0, np.nan)
    .div(60))
pivot

fish_id          cs-ii-29B                                         \
unique_run_id cs-ii-29_395 cs-ii-29_396 cs-ii-29_399 cs-ii-33_188   
bout_time_bin                                                       
(0, 60]           0.050000     0.016667     0.016667          NaN   
(60, 120]              NaN     0.016667     0.016667          NaN   
(120, 180]             NaN          NaN     0.016667          NaN   
(180, 240]        0.016667          NaN          NaN          NaN   
(240, 300]             NaN          NaN     0.016667          NaN   
(300, 360]             NaN          NaN          NaN          NaN   
(360, 420]             NaN     0.033333          NaN          NaN   
(420, 480]             NaN          NaN          NaN          NaN   
(480, 540]             NaN          NaN          NaN          NaN   
(540, 600]             NaN     0.050000          NaN          NaN   
(600, 660]             NaN          NaN     0.016667          NaN   
(660, 720]             NaN     0.033333          NaN          NaN   
(720, 780]             NaN     0.100000          NaN          NaN   
(780, 840]             NaN     0.016667     0.033333          NaN   
(840, 900]             NaN          NaN          NaN          NaN   
(900, 960]             NaN          NaN          NaN          NaN   
(960, 1020]            NaN          NaN          NaN          NaN   
(1020, 1080]           NaN          NaN          NaN          NaN   
(1080, 1140]           NaN          NaN          NaN          NaN   
(1140, 1200]           NaN          NaN          NaN          NaN   
(1200, 1260]           NaN          NaN          NaN          NaN   
(1260, 1320]           NaN          NaN          NaN          NaN   
(1320, 1380]           NaN          NaN          NaN          NaN   
(1380, 1440]           NaN          NaN          NaN          NaN   
(1440, 1500]           NaN          NaN          NaN          NaN   
(1500, 1560]           NaN          NaN          NaN          NaN   
(1560, 1620]           NaN          NaN          NaN          NaN   
(1620, 1680]           NaN          NaN          NaN          NaN   
(1680, 1740]           NaN          NaN          NaN          NaN   
(1740, 1800]           NaN          NaN          NaN          NaN   

fish_id                                                            \
unique_run_id cs-ii-33_189 cs-ii-33_195 cs-ii-34_202 cs-ii-34_203   
bout_time_bin                                                       
(0, 60]                NaN          NaN          NaN          NaN   
(60, 120]              NaN          NaN          NaN          NaN   
(120, 180]             NaN          NaN          NaN          NaN   
(180, 240]             NaN          NaN          NaN          NaN   
(240, 300]             NaN          NaN          NaN          NaN   
(300, 360]             NaN          NaN          NaN          NaN   
(360, 420]             NaN          NaN          NaN          NaN   
(420, 480]             NaN          NaN          NaN          NaN   
(480, 540]             NaN          NaN          NaN          NaN   
(540, 600]             NaN          NaN          NaN          NaN   
(600, 660]             NaN          NaN          NaN          NaN   
(660, 720]             NaN          NaN          NaN          NaN   
(720, 780]             NaN          NaN          NaN          NaN   
(780, 840]             NaN          NaN          NaN          NaN   
(840, 900]             NaN          NaN          NaN          NaN   
(900, 960]             NaN          NaN          NaN          NaN   
(960, 1020]            NaN          NaN          NaN          NaN   
(1020, 1080]           NaN          NaN          NaN          NaN   
(1080, 1140]           NaN          NaN          NaN          NaN   
(1140, 1200]           NaN          NaN          NaN          NaN   
(1200, 1260]           NaN          NaN          NaN          NaN   
(1260, 1320]           NaN          NaN          NaN          NaN 

In [8]:
(pivot
 .melt(
    value_name='bout_frequency',
    ignore_index=False)
 .dropna())

,fish_id,unique_run_id,bout_frequency
bout_time_bin,,,
"(0, 60]",cs-ii-29B,cs-ii-29_395,0.050000
"(180, 240]",cs-ii-29B,cs-ii-29_395,0.016667
"(0, 60]",cs-ii-29B,cs-ii-29_396,0.016667
"(60, 120]",cs-ii-29B,cs-ii-29_396,0.016667
"(360, 420]",cs-ii-29B,cs-ii-29_396,0.033333
...,...,...,...
"(840, 900]",cs-ii-34A,cs-ii-34_209,0.183333
"(900, 960]",cs-ii-34A,cs-ii-34_209,0.516667
"(960, 1020]",cs-ii-34A,cs-ii-34_209,0.150000
